In [1]:
from sklearn.metrics.pairwise import cosine_similarity
def collaborative_filtering(cluster, df, user):
  df=df.groupby(['user_id','name'])['stars_x'].mean().reset_index()
  #User-Item similarity matrix 
  pivoted = df.pivot(index='user_id',columns='name',values='stars_x')
  pivoted.columns.name=None
  pivoted_df=pivoted

  #User-Item similarity matrix fill na with 0
  pivoted_df=pivoted_df.fillna(0)

  #cosine similarity between user-user similarity matrix 
  cos_mat=cosine_similarity(pivoted_df, pivoted_df)
  np.fill_diagonal(cos_mat, -np.inf)

  cos_df = pd.DataFrame(cos_mat)
  # find the maximum values of each row to find max similarity among users
  maxValues = cos_df.idxmax(axis=1).reset_index()
  maxValues.rename(columns = {'index':'TargetUser',0:'MostSimilarUser'}, inplace = True)
  maxValues['TargetUser'] = pivoted_df.index
  maxValues['MostSimilarUser']=pivoted_df.index[maxValues['MostSimilarUser']]

  user_max_rating=pivoted_df.idxmax(axis=1).reset_index()
  user_max_rating.rename(columns = {0:'RestaurantToRecommend','user_id':'MostSimilarUser'}, inplace = True)
  maxValues=pd.merge(user_max_rating,maxValues,on='MostSimilarUser',how='inner')
  return(list(maxValues[maxValues['TargetUser'] == user].RestaurantToRecommend))

def recommend_sentiment(cluster, niche):
  niche_clus=niche[niche['clusters']==cluster]
  niche_clus=niche_clus[niche_clus['sentiment']==1]
  return(list(niche_clus[niche_clus['stars_x']==niche_clus['stars_x'].max()].name))

In [2]:
import pandas as pd
import numpy as np

def initiate_recommend(user):
    bot10_sent=pd.read_csv("/Users/linalouis/Downloads/niche_sentiment_dt_clf.csv") 
    bot10_sent = bot10_sent[['name','stars_x','clusters','sentiment']]
    top10=pd.read_csv("/Users/linalouis/Downloads/top10_with_cluster.csv")


    #get cluster

    cluster=list(top10[top10['user_id']==user].clusters)
    cluster=cluster[0]

    #get recommendation
    top10 = top10.loc[(top10['clusters'] ==cluster)]
    top10=top10[["user_id",'stars_x','name']]
    top10.reset_index()

    recommend=[]
    recommend.append(collaborative_filtering(cluster, top10, user))
    
    print(user)

    recommend.append(recommend_sentiment(cluster, bot10_sent))
    #recommend = [item for sublist in recommend for item in sublist]
    print(recommend)
    return(recommend)

In [3]:
import tkinter as tk
# from tkinter import Canvas, Entry, Grid, INSERT, END
import pandas as pd
from tkinter.messagebox import showinfo
from PIL import Image, ImageTk
# import numpy as np

# take the data


window = tk.Tk()

#setting tkinter window size
width= window.winfo_screenwidth()
height= window.winfo_screenheight()
window.geometry("%dx%d" % (width, height))
window.title("Restaurant Recommender")

#Read CSV File 
df = pd.read_csv('/Users/linalouis/Downloads/top10_with_cluster.csv')

#Fetching top 20 user_id
lst = df['user_id'].head(20)
total_rows = len(lst)
total_columns = len(lst[0])

#Left Frame
frame1 = tk.Frame(master=window, width=400, bg="#135D75")
frame1.pack(fill="both", side=tk.LEFT)

#Main Heading label1
label1 = tk.Label(master=frame1, text="Enter USER ID:")
label1.place(x=30,y=50)
label1.config(font='Helvetica 18')
label1.config(bg="#135D75",fg="white")

#ID Input
user_id=tk.Entry(master=frame1, width = 35)
user_id.place(x = 170, y = 50, width = 200)
 
#Header after the label1
horizontal =tk.Frame(master=frame1, bg='white', height=1,width=300)
horizontal.place(x=30, y=90)

#Table Grid Frame
framegrid = tk.Frame(master=frame1, width=400, height=900, bg="#135D75", borderwidth=0)
for x, i in zip(lst,range(total_rows)):
    for y in range(1):
        #Start count from 1
        i = i+1

        #Text to display in each label
        txt = (str(i) + " : " +  x)

        #Setting padding and position of the grid
        framegrid.grid(row=i, column=y, padx=5, pady=5)

        #Passing each user_id in each label
        labelgrid = tk.Label(master=framegrid, text=txt, bg="#135D75",fg="white")
        labelgrid.config(font='Helvetica 14')
        labelgrid.pack(pady=1,side="top", anchor="w")
        
#Placing table grid
framegrid.place(x=30, y=130)       

#Right Frame
frame2 = tk.Frame(master=window, bg="#F0F0F0")
frame2.place(relx=0.5, rely=0.5,anchor="center")
frame2.pack(expand = True,fill="both", side=tk.LEFT)

images = []

image = Image.open("/Users/linalouis/Downloads/img.png")
resize_image = image.resize((1000, 400))
img = ImageTk.PhotoImage(resize_image)
images.append(img) 

#Header after the label1
frame2_child =tk.Frame(master=frame2, bg='#F0F0F0', height=200, width=200)
frame2_child.place(relx=0.5, rely=0.3,anchor="center")

def screen2():
    global ws2
    ws2 = tk.Toplevel(window)
    ws2.title('Restaurant Recommendations')
    ws2.geometry('1000x1000')
    ws2.config(bg='#135D75')
    user=user_id.get()
    print(user)
    recommend=initiate_recommend(user)
    i=2
    label1=tk.Label(master=ws2, text="Restaurants that best fit your preferences!", bg='yellow')
    label1.grid(row=1, column=1, pady=10, padx=20)
    for r in recommend[0]:
        out=tk.Label(master=ws2, text=r)
        out.grid(row=i, column=1, pady=10, padx=20)
        i=i+1
    label2=tk.Label(master=ws2, text="Niche Restaurants that you would love!", bg='yellow')
    label2.grid(row=i, column=1, pady=10, padx=20)
    i=i+1
    for r in recommend[1]:
        out=tk.Label(master=ws2, text=r)
        out.grid(row=i, column=1, pady=10, padx=20)
        i=i+1

    
#Main Heading label1
label1 = tk.Label(
    master=frame2_child, 
    text="Get your recommendation",
    bg="#F0F0F0",
    fg="#135D75"
)
label1.place(relx=0.4, rely=0.3,anchor="center")
label1.config(font='Helvetica 18')
label1.pack(pady=20,side="top", anchor="w")

submit_btn = tk.Button(
    frame2_child,
    text='Submit',
    height=1,
    width=15,
    bg="#135D75",
    fg="black",
    command=screen2
)

submit_btn.place(y=200)   

submit_btn.config(font='Helvetica 14')

submit_btn.pack(
    ipadx=5,
    ipady=5,
    expand=True
)

img1 = tk.Label(frame2, image = img)
img1.place(x=100,y=50)
img1.pack(side="bottom")


#Setting the window resizing to false
window.resizable(False, False)

#Recurrsive loop to keep running the application
window.mainloop()

U8GjhUk-vue80lyUYY7eyA
U8GjhUk-vue80lyUYY7eyA
[["Boathouse at Hendry's Beach"], ['Hotdog Mini Mart', 'Broad Street Oyster Company']]


In [ ]:
df['user_id'].unique()